# Create spectral-temporal metrics and visualize them in ipynb
#### Input from *fitoprincipe* @ github
https://github.com/baumi-berlin/ee-jupyter-contrib/blob/master/examples/getting-started/display-interactive-map.ipynb

In [ ]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json, osr

# Load a polygon shapefile for the extent
here, we load a shapefile for the extent that we want to process only. This reduces computional time, and also will directly zoom to the location we are working. Test-case is a tile in the Chaco

In [ ]:
shp = baumiVT.CopyToMem("C:/Miniconda3/envs/py37/Lib/site-packages/baumiTools/geeTools/Chiqui_Outline.shp")
lyr = shp.GetLayer()
featList = []
feat = lyr.GetNextFeature()
geom = feat.GetGeometryRef()

# Convert the CS to EPSG:4326
source_SR = geom.GetSpatialReference()
target_SR = osr.SpatialReference()
target_SR.ImportFromEPSG(4326)
trans = osr.CoordinateTransformation(source_SR, target_SR)
geom.Transform(trans)

geom_json = json.loads(geom.ExportToJson())
geom_coord = geom_json['coordinates']
geom_EE = ee.Geometry.Polygon(coords=geom_coord)
lyr.ResetReading()

### Somehow, this only works for the Sentinel-2 metrics. I assume that there are too many L8 iamgery being selected. For the Path/Row selection the function works smoothly (EDIT: when I run it with a reduced selection of imagery (e.g., only L5/7) then a composite emerges

# Define the composite/metric we want to look at

In [ ]:
median_L8 = baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=1, endMonth=3, roi=geom_EE)# path=228, row=75)#, roi=geom_EE)# additional arguments: roi=feat_EE or path=227, row=28
#for win in [[4,6], [7,9],[10,12]]:
#    median_L8 = median_L8.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=win[0], endMonth=win[1]))
bandNames_L8 = median_L8.bandNames()
#bandNames_L8.getInfo()
median_S2 = baumiEE.Calculate_S2_Seasonal_Median(year=2019, startMonth=1, endMonth=3, roi=geom_EE)
bandNames_S2 = median_S2.bandNames()
#bandNames_S2.getInfo()

# Display a Dynamic Map
To display an Earth Engine generated tiles on the interactive map, we will use ipyleaflet's TileLayer object. First we start by defining a function that can generate a tile layer URL from an Earth Engine image object.

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [ ]:
Map = ipyleaflet.Map(center=(-14.438411, -63.743799), zoom=4,
                      layout={'height':'600px'},
                      scroll_wheel_zoom=True,
                    show_loading=True)
Landsat8 = ipyleaflet.TileLayer(name="Landsat8", url=GetTileLayerUrl(
    median_L8.visualize(min=0, max=8000, gamma=1.5,bands= ['R_2019-01-03', 'G_2019-01-03', 'B_2019-01-03'])))
Sentinel2 = ipyleaflet.TileLayer(name="Sentinel2", url=GetTileLayerUrl(
    median_S2.visualize(min=0, max=8000, gamma=1.5,bands= ['R_2019-01-03', 'G_2019-01-03', 'B_2019-01-03'])))

Map.add_layer(Landsat8)
Map.add_layer(Sentinel2)
Map.add_control(ipyleaflet.LayersControl(position='topright'))

Map
